In [296]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Dicionário de TAGS do XML

### - cProd : Código do Fornecedor
### - cEAN : EAN do Produto, em alguns casos é o EAN da Caixa do Produto
### - xProd: Nome do Produto
### - NCM: Código NCM do produto
### - vUnCom: Valor unitário do produto
### - uCom : Unidade comercializada
### - qCom : Quantidade comercializada
### - CST : Código de Situação Tributária, define quais produtos possuem substituuição tributária: CSOSN: 201 - ST Simples Nacional e CST 10 - ST Regime Normal.
### - vICMSST: Valor da ST.
### - pICMS: Alíquota ICMS
### - vIPI: Define o valor do IPI.

In [297]:
import xml.etree.ElementTree as ET
from xml.dom import minidom

In [298]:
xml = open('/content/drive/MyDrive/Python + XML/XMLS/fortlev.xml')
nfe = minidom.parse(xml)

In [299]:
num_nfe = nfe.getElementsByTagName('nNF')
n_NFE = num_nfe[0].firstChild.data
print(n_NFE)

1432797


In [300]:
#Funções 

#Função que varre dados do tipo text do xml e insere em uma lista de string.
def nodeText(tag,nfe):

  dados = []
  item = nfe.getElementsByTagName(tag)

  for x in item:
    dados.append(x.firstChild.data)
  
  return dados

#Função que varre dados do tipo number do xml e insere em uma lista de float.
def nodeNumber(tag,nfe):

  dados = []
  item = nfe.getElementsByTagName(tag)

  for x in item:
    dados.append(float(x.firstChild.data))

  return dados

#Função que completa com o 0 as posições do produtos sem substituição tributária

def listaST(lista1,lista2):

  a = len(lista1) - len(lista2)

  while(a != 0):
    lista2.append(0)
    a-=1

  x = 0
  y = 0
  subsTrib = []
  
  while(x < len(lista1)):
    if lista1[x] == '201' or lista1[x] == '10':
      subsTrib.append(float(lista2[y]))
      y+=1
    else:
      subsTrib.append(float(0))
    x+=1

  return subsTrib

#Criando a lista com o CST apenas da Substituição Tributátia (ST)
#Aqui o no ICMS é pai do CST

def cstList(lista):
  
  x = 0
  inter = []

  while(x<len(lista)):
    a = lista[x]
    inter.append(a)
    x+=4

  return inter

#Calcula o preço de venda

from math import floor
def precoVenda(lista,margem):

  custo = []
  venda = []
  x = 0
  while(x < len(lista)):
    y = (lista[x])/margem
    custo.append(round(y,2))
    
    j = floor(custo[x])
    y = round(j)
    if (j - y) < 1:
      j = j + (0.99 - (j - y))
      venda.append(round(j,2))
      
    x += 1

  return venda

#Ajustando IPI dos produtos

def ipiLista(lista):
  x = 1
  inter = []

  while(x<len(lista)):
    a = lista[x]
    inter.append(a)
    x+=4

  return inter

def listaIPI(lista1,lista2):

  a = len(lista1) - len(lista2)

  while(a != 0):
    lista2.append(0)
    a-=1

  x = 0
  y = 0
  ipi = []
  
  while(x < len(lista1)):
    if lista1[x] == '50':
      ipi.append(float(lista2[y]))
      y+=1
    else:
      ipi.append(float(0))
    x+=1

  return ipi


In [301]:
#Encontrando o Regime Tributário 

item = nfe.getElementsByTagName('CSOSN')

if len(item) != 0:
  codTributacao = nodeText('CSOSN',nfe)
  subs = nodeText('vICMSST',nfe)
else:
  codTributacao = nodeText('CST',nfe)
  codTributacao = cstList(codTributacao)
  subs = nodeText('vICMSST',nfe)

In [302]:
#Teste da Chamada da Função
x = nodeText('cEAN',nfe)
subs

['18.71',
 '8.20',
 '100.15',
 '36.09',
 '79.19',
 '14.18',
 '24.75',
 '18.39',
 '6.53',
 '10.94',
 '5.77',
 '19.36',
 '33.26',
 '19.20',
 '18.38',
 '127.96',
 '155.90',
 '621.26']

In [303]:
#Teste Chamada função do IPI
cst = nodeText('CST',nfe)
ipi = nodeNumber('vIPI',nfe)
cstIpi = ipiLista(cst)
listaIPI = listaIPI(cstIpi,ipi[:-1])
listaIPI

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [304]:
#Fazendo algumas verificações:

#Quantidade de itens NFE

qt_itens = len(nodeText('xProd',nfe))
qt_itens

19

In [305]:
#Criação do DataFrame

import pandas as pd

df = pd.DataFrame()
df['EAN'] = nodeText('cEAN',nfe)
df['Produto'] = nodeText('xProd',nfe)
df['Quantidade'] = nodeNumber('qCom',nfe)
df['Valor UNIT'] = nodeNumber('vUnCom',nfe)
df['NCM'] = nodeText('NCM',nfe)
valorT = nodeNumber('vProd',nfe)
df['Valor Total'] = valorT[:-1]

In [306]:
st = listaST(codTributacao,subs)
st
if qt_itens == len(st):
  df['ST'] = st
else:
  df['ST'] = 0

In [307]:
ipi = nodeNumber('vIPI',nfe)
ipi
if qt_itens == len(listaIPI):
  df['IPI'] = listaIPI
else:
  df['IPI'] = 0

In [310]:
df['Custo Final'] = df['Valor UNIT'] + ((df['ST']+df['IPI'])/df['Quantidade'])
margem = float(input('Insira a margem de lucro: '))
df['Preço de Venda'] = precoVenda(list(df['Custo Final']),margem)
df

Insira a margem de lucro: 0.65


,EAN,Produto,Quantidade,Valor UNIT,NCM,Valor Total,ST,IPI,Custo Final,Preço de Venda
0,7898543590194,TE ESG SN DN 50 FORTLEV,50.0,2.622000,39174090,131.10,18.71,0.0,2.996200,4.99
1,7898543590170,TE ESG SN DN 40 FORTLEV,50.0,1.148800,39174090,57.44,8.20,0.0,1.312800,2.99
2,7898543592679,TE ESG SN DN 100 FORTLEV,100.0,7.017800,39174090,701.78,100.15,0.0,8.019300,12.99
3,7898543590910,LUVA ESG SN DN 100 FORTLEV,100.0,2.528600,39174090,252.86,36.09,0.0,2.889500,4.99
4,7898911343384,JOELHO 90 ESG 100 FORTLEV,200.0,2.774400,39174090,554.88,79.19,0.0,3.170350,4.99
5,7898543590446,TE SOLDAVEL 50MM FORTLEV,20.0,4.968000,39174090,99.36,14.18,0.0,5.677000,8.99
6,7898911343735,LUVA DE CORRER SOLD 25MM FORTLEV,24.0,7.225833,39174090,173.42,24.75,0.0,8.257083,12.99
7,7898911343711,LUVA DE CORRER SOLD 20MM FORTLEV,24.0,5.369583,39174090,128.87,18.39,0.0,6.135833,9.99
8,7898543590996,JOELHO 90 SOLDAVEL 50MM FORTLEV,20.0,2.288500,39174090,45.77,6.53,0.0,2.615000,4.99
9,7898543590989,JOELHO 90 SOLDAVEL 25MM FORTLEV,200.0,0.383200,39174090,76.64,10.94,0.0,0.437900,0.99


In [309]:
#Gerando planilha no excel

df.to_excel('precos.xlsx',sheet_name= f'NFE {n_NFE}',index = False)